In [17]:
# import backoff  # for exponential backoff
import pandas as pd
import seaborn as sns
# import random
import matplotlib.pyplot as plt
import pprint
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial import distance
from scipy.spatial import distance
from sentence_similarity import sentence_similarity

import json
from sentence_transformers import SentenceTransformer
import ast
from pymilvus import (
    connections,
    utility,
    Collection,
)
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
from transformers import AutoTokenizer

import seaborn as sns
import matplotlib.pyplot as plt

In [18]:
sentences = ["This is an example sentence", "Each sentence is converted"]
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(sentences)
print(len(embeddings))

2


In [19]:
# Assuming you have a JSON file named 'your_file.json'
file_path = 'C:\\Users\\mateo\\Documents\\Universidad\\Tesis\\tesis-qa-system\\evaluations\\question_bank.json'
# Read the JSON file and convert it to a DataFrame
df = pd.read_json(file_path)
# Display the DataFrame
df

,question,answer
0,What are the three main types of pollution?,"Air pollution, water pollution, and land pollu..."
1,What is the leading cause of air pollution?,Burning fossil fuels
2,What is the most common type of water pollution?,Sewage
3,What is the most common type of land pollution?,Solid waste
4,What are the main effects of air pollution?,"Health problems, climate change, and acid rain"
5,What are the main effects of water pollution?,"Health problems, ecosystem damage, and economi..."
6,What are the main effects of land pollution?,"Health problems, ecosystem damage, and environ..."
7,What are the main sources of air pollution?,"Transportation, industry, and power plants"
8,What are the main sources of water pollution?,"Sewage, agriculture, and industry"
9,What are the main sources of land pollution?,"Solid waste, mining, and agriculture"


In [20]:
df.shape

(50, 2)

In [21]:
def get_retreiver(question):

    connections.connect("default", host="localhost", port="19530")
    collection_name = 'prueba_final_3'
    collection = Collection(name=collection_name)
    collection.load()

    model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2", device='cpu')

    question_vector = model.encode([question])[0].tolist()

    search_params = {
        "metric_type": "COSINE", 
        "offset": 0, 
        "ignore_growing": False
    }
    search_result = collection.search(
        data=[question_vector], 
        anns_field='embedding', 
        param=search_params, 
        limit=2,
        output_fields=['metadata'],
    )

    import re
    print(search_result)
    print(type(search_result))
    hits = search_result
    array = []
    for hits in iter(search_result):
        data_list = [hit.entity.get('metadata') for hit in hits]
    raw_answer = [answer['split'] for answer in data_list]

    raw_answer_final = '. '.join(raw_answer)

    return raw_answer_final
        

In [22]:
retreiver_anwers = []

for i in range(len(df)):

    question = df.loc[i, 'question']

    retreiver_ans = get_retreiver(question)

    retreiver_anwers.append(retreiver_ans) 

df['answer_retreiver'] = retreiver_anwers




['["id: 446616421319185884, distance: 0.6478539109230042, entity: {\'metadata\': {\'id\': 46940, \'PageURL\': \'https://en.wikipedia.org/wiki/Water_pollution\', \'Title\': \'Water pollution\', \'CreationDate\': \'2023-12-16T18:40:07Z\', \'Seccion\': nan, \'Subseccion\': nan, \'Contenido\': \'Water pollution (or aquatic pollution) is the contamination of water bodies, usually as a result of human activities, so that it negatively affects its uses.:\\\\u200a6\\\\u200a Water bodies include lakes, rivers, oceans, aquifers, reservoirs and groundwater. Water pollution results when contaminants mix with these water bodies. Contaminants can come from one of four main sources: sewage discharges, industrial activities, agricultural activities, and urban runoff including stormwater. Water pollution is either surface water pollution or groundwater pollution. This form of pollution can lead to many problems, such as the degradation of aquatic ecosystems or spreading water-borne diseases when people

In [23]:
df.head()

,question,answer,answer_retreiver
0,What are the three main types of pollution?,"Air pollution, water pollution, and land pollu...",Water pollution (or aquatic pollution) is the ...
1,What is the leading cause of air pollution?,Burning fossil fuels,Air pollution is the contamination of air due ...
2,What is the most common type of water pollution?,Sewage,Water pollution (or aquatic pollution) is the ...
3,What is the most common type of land pollution?,Solid waste,"Soil contamination, soil pollution, or land po..."
4,What are the main effects of air pollution?,"Health problems, climate change, and acid rain",Even at levels lower than those considered saf...


#### LLAMA2

In [24]:
import functools

In [25]:
import re
def inicitalize_model(model_path_value):
    lcpp_llm = Llama(
        model_path=model_path_value,
        n_threads=11, # CPU cores
        n_batch=1024, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
        n_gpu_layers=32, # Change this value based on your model and your GPU VRAM pool.
        n_ctx=3072,
        )
    return lcpp_llm

def llama_with_context(lcpp_llm, topic_value, prompt_value,
                       context_value, prompt_template_value, max_tokens_value,
                       temperature_value, top_p_value, repeat_penalty_value,
                       top_k_value, echo_value):

    #See the number of layers in GPU
    #lcpp_llm.params.n_gpu_layers

    topic = topic_value

    prompt = prompt_value

    context = context_value

    prompt_template = prompt_template_value

    #print(type(prompt_template))

    response=lcpp_llm(prompt=prompt_template,
                    max_tokens= max_tokens_value,
                    temperature= temperature_value,
                    top_p=top_p_value,
                    repeat_penalty=repeat_penalty_value,
                    top_k = top_k_value,
                    echo= echo_value)
    prueba_respuesta = response["choices"][0]["text"]
    pattern = r'\[INST\].*?\[/INST\]\s*(.*)'
    match = re.search(pattern, prueba_respuesta, re.DOTALL)
    if match:
        extracted_content = match.group(1)
        print(extracted_content)
        response = extracted_content
        return response
    else:
        print("No se encontró el contenido deseado.")
        print(prueba_respuesta)
        return prueba_respuesta
    


In [26]:
llama_answers = []
retreiver_anwers = []

df = df
print(df.shape)

model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)
lcpp_llm = inicitalize_model(model_path)

for i in range(len(df)):
    print("iteracion ", str(i))

    question = df.loc[i, 'question']

    retreiver_ans = get_retreiver(question)

    retreiver_anwers.append(retreiver_ans) 

    #Prompt (Pregunta al modelo)
    prompt = question
    #Context
    context = retreiver_ans
    topic = 'Contamination'
    #template
    # 3.9 min de respuesta
#     prompt_template=f'''[INST] <<SYS>>
# You are a helpful, respectful and honest assistant. Always answer briefly and concisely, without introductory sentences and be helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
# also you are a {topic} expert, answer questions about {topic} as best as you can with this context {context}.
# </SYS>>\n{prompt}[/INST]'''
    # 4.8 min de respuesta
#     prompt_template=f'''[INST] <<SYS>>
# You are a helpful, respectful and honest assistant be helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
# also you are a {topic} expert, answer questions about {topic} as best as you can with this context {context}.
# </SYS>>\n{prompt}[/INST]'''
    
    prompt_template=f'''[INST] <<SYS>>
You are an assistant that always answer concisely, without introductory sentences and be helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
also you are a {topic} expert, answer questions about {topic} as best as you can with this context {context}.
</SYS>>\n{prompt}[/INST]'''
    max_tokens=2500
    temperature=0.1
    top_p=0.95  
    repeat_penalty=1.2
    top_k=100
    echo=True

    respuesta = llama_with_context(lcpp_llm, topic, prompt, context, prompt_template, max_tokens,  temperature, top_p, repeat_penalty, top_k, echo)
    

    llama_answers.append(respuesta)

df['retreiver_answers12'] = retreiver_anwers
df['qasystem_answers'] = llama_answers

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 


iteracion  0
['["id: 446616421319185884, distance: 0.6478539109230042, entity: {\'metadata\': {\'id\': 46940, \'PageURL\': \'https://en.wikipedia.org/wiki/Water_pollution\', \'Title\': \'Water pollution\', \'CreationDate\': \'2023-12-16T18:40:07Z\', \'Seccion\': nan, \'Subseccion\': nan, \'Contenido\': \'Water pollution (or aquatic pollution) is the contamination of water bodies, usually as a result of human activities, so that it negatively affects its uses.:\\\\u200a6\\\\u200a Water bodies include lakes, rivers, oceans, aquifers, reservoirs and groundwater. Water pollution results when contaminants mix with these water bodies. Contaminants can come from one of four main sources: sewage discharges, industrial activities, agricultural activities, and urban runoff including stormwater. Water pollution is either surface water pollution or groundwater pollution. This form of pollution can lead to many problems, such as the degradation of aquatic ecosystems or spreading water-borne disease

In [ ]:
df.head()


,question,answer,answer_retreiver,retreiver_answers12,qasystem_answers
0,What are the three main types of pollution?,"Air pollution, water pollution, and land pollu...",Water pollution (or aquatic pollution) is the ...,Water pollution (or aquatic pollution) is the ...,The three main types of pollution are:\n\n1. S...
1,What is the leading cause of air pollution?,Burning fossil fuels,Air pollution is the contamination of air due ...,Air pollution is the contamination of air due ...,The leading cause of air pollution is fuel com...
2,What is the most common type of water pollution?,Sewage,Water pollution (or aquatic pollution) is the ...,Water pollution (or aquatic pollution) is the ...,The most common type of water pollution is sur...


In [ ]:
df.to_csv('C:\\Users\\mateo\\Documents\\Universidad\\Tesis\\tesis-qa-system\\evaluations\\pruebasPaSotoPosibleCompleta.csv', index=False)